# Radiomics Feature Extraction Tutorial

This notebook demonstrates how to use the `onem_radiomics` module to extract comprehensive radiomics features from medical images and masks.

## 📋 Table of Contents
1. [Setup and Imports](#setup)
2. [Single Image Feature Extraction](#single)
3. [Batch Processing](#batch)
4. [Feature Analysis](#analysis)
5. [Configuration Management](#config)
6. [Visualization](#visualization)

## 🔧 Setup and Imports {#setup}

In [ ]:
# Core imports
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add project root to path
project_root = Path().absolute().parent
sys.path.append(str(project_root))

# Import onem_radiomics modules
from onem_radiomics import RadiomicsExtractor
from onem_radiomics.config.settings import get_preset_config
from onem_radiomics.utils.radiomics_utils import analyze_features

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ All modules imported successfully!")
print(f"Project root: {project_root}")

## 📊 Single Image Feature Extraction {#single}

In [ ]:
# Initialize the radiomics extractor
extractor = RadiomicsExtractor()
print("🔬 Radiomics extractor initialized")

# Example paths (replace with your actual file paths)
image_path = "sample_data/patient001_ct.nii.gz"
mask_path = "sample_data/patient001_mask.nii.gz"

# Check if sample data exists
if not os.path.exists(image_path):
    print(f"⚠️  Sample image not found: {image_path}")
    print("Please replace with your actual NIfTI file paths")
else:
    # Extract features with default CT configuration
    print("🚀 Extracting features...")
    features = extractor.extract_features(
        image_path=image_path,
        mask_path=mask_path,
        config_name='ct_lung'  # Use CT lung cancer preset
    )
    
    print(f"✅ Extracted {len(features)} features")
    print("\n📋 Sample features:")
    for i, (key, value) in enumerate(list(features.items())[:10]):
        print(f"  {i+1:2d}. {key}: {value:.4f}")
    
    if len(features) > 10:
        print(f"  ... and {len(features) - 10} more features")

## 🔄 Batch Processing {#batch}

In [ ]:
# Batch processing setup
image_dir = "sample_data/images/"
mask_dir = "sample_data/masks/"
output_csv = "output/radiomics_features.csv"

# Create output directory if it doesn't exist
os.makedirs(os.path.dirname(output_csv), exist_ok=True)

# Check if directories exist
if os.path.exists(image_dir) and os.path.exists(mask_dir):
    print(f"📁 Processing images from: {image_dir}")
    print(f"📁 Processing masks from: {mask_dir}")
    
    # Extract features for all images in the directories
    results = extractor.extract_batch(
        image_dir=image_dir,
        mask_dir=mask_dir,
        output_csv=output_csv,
        config_name='ct_lung',
        parallel=True,  # Enable parallel processing
        n_workers=4     # Number of parallel workers
    )
    
    print(f"✅ Batch processing completed!")
    print(f"📊 Results saved to: {output_csv}")
    
    # Load and display results
    df = pd.read_csv(output_csv)
    print(f"\n📈 Processed {len(df)} cases")
    print(f"📋 Feature columns: {len(df.columns) - 1}")  # -1 for ID column
    
    # Display first few rows
    print("\n👀 Sample results:")
    display(df.head())
else:
    print(f"⚠️  Sample directories not found:")
    print(f"   Images: {image_dir}")
    print(f"   Masks: {mask_dir}")
    print("Please replace with your actual directory paths")

## 📈 Feature Analysis {#analysis}

In [ ]:
# If we have results from batch processing, analyze them
if 'df' in locals():
    print("🔍 Performing feature analysis...")
    
    # Get feature columns (excluding ID column)
    feature_cols = [col for col in df.columns if col != 'PatientID']
    
    # Basic statistics
    print("\n📊 Feature Statistics:")
    stats_df = df[feature_cols].describe().T
    display(stats_df[['mean', 'std', 'min', 'max']].head(10))
    
    # Feature correlation analysis
    print("\n🔗 Computing feature correlations...")
    correlation_matrix = df[feature_cols].corr()
    
    # Find highly correlated features
    high_corr_pairs = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            corr_val = correlation_matrix.iloc[i, j]
            if abs(corr_val) > 0.9:  # High correlation threshold
                high_corr_pairs.append((
                    correlation_matrix.columns[i],
                    correlation_matrix.columns[j],
                    corr_val
                ))
    
    print(f"\n⚠️  Found {len(high_corr_pairs)} highly correlated feature pairs (|r| > 0.9):")
    for i, (feat1, feat2, corr) in enumerate(high_corr_pairs[:10]):
        print(f"  {i+1:2d}. {feat1} ↔ {feat2}: r = {corr:.3f}")
    
    if len(high_corr_pairs) > 10:
        print(f"  ... and {len(high_corr_pairs) - 10} more pairs")
else:
    print("⚠️  No batch processing results available for analysis")
    print("Run the batch processing cell first to generate data for analysis")

## ⚙️ Configuration Management {#config}

In [ ]:
# Explore available preset configurations
print("📋 Available preset configurations:")
preset_configs = [
    'default', 'ct_lung', 'ct_brain', 'mri_brain', 
    'pet_tumor', 'research', 'production'
]

for config_name in preset_configs:
    try:
        config = get_preset_config(config_name)
        print(f"\n🔧 {config_name}:")
        print(f"  - Image type: {config.get('image_type', 'unknown')}")
        print(f"  - Feature types: {', '.join(config.get('feature_types', []))}")
        print(f"  - Bin width: {config.get('bin_width', 'N/A')}")
        print(f"  - Resampling: {config.get('resampling', 'disabled')}")
    except Exception as e:
        print(f"  ⚠️  Error loading {config_name}: {e}")

# Create custom configuration
print("\n🎨 Creating custom configuration...")
custom_config = {
    'image_type': 'CT',
    'feature_types': ['firstorder', 'texture'],
    'bin_width': 20,
    'resampling': {
        'voxel_size': [1.0, 1.0, 3.0],  # Isotropic voxels
        'interpolator': 'sitkBSpline'
    },
    'preprocessing': {
        'normalize': True,
        'remove_outliers': True
    },
    'texture_settings': {
        'glcm_distances': [1, 2, 3],
        'glrlm_distances': [1, 2, 3]
    }
}

print("✅ Custom configuration created")
print(f"📋 Custom config: {custom_config}")

## 📊 Visualization {#visualization}

In [ ]:
# If we have results, create visualizations
if 'df' in locals() and len(df) > 1:
    feature_cols = [col for col in df.columns if col != 'PatientID']
    
    # 1. Feature distribution plot
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle('Feature Distributions', fontsize=16, fontweight='bold')
    
    # Plot first 6 features
    for i, feature in enumerate(feature_cols[:6]):
        row, col = i // 3, i % 3
        axes[row, col].hist(df[feature].dropna(), bins=20, alpha=0.7, edgecolor='black')
        axes[row, col].set_title(feature)
        axes[row, col].set_xlabel('Value')
        axes[row, col].set_ylabel('Frequency')
        axes[row, col].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 2. Correlation heatmap (subset)
    plt.figure(figsize=(12, 8))
    correlation_subset = df[feature_cols[:15]].corr()  # First 15 features
    
    mask = np.triu(np.ones_like(correlation_subset, dtype=bool))
    sns.heatmap(correlation_subset, mask=mask, annot=True, cmap='coolwarm', 
                center=0, square=True, fmt='.2f', cbar_kws={"shrink": .8})
    plt.title('Feature Correlation Heatmap (First 15 Features)', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # 3. Box plot for selected features
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    fig.suptitle('Feature Box Plots', fontsize=16, fontweight='bold')
    
    # First-order features
    first_order_features = [col for col in feature_cols 
                           if any(keyword in col.lower() 
                                 for keyword in ['mean', 'median', 'std', 'skewness', 'kurtosis'])][:4]
    
    if first_order_features:
        df[first_order_features].boxplot(ax=axes[0])
        axes[0].set_title('First-Order Features')
        axes[0].tick_params(axis='x', rotation=45)
    
    # Texture features
    texture_features = [col for col in feature_cols 
                        if any(keyword in col.lower() 
                              for keyword in ['glcm', 'glrlm', 'glszm'])][:4]
    
    if texture_features:
        df[texture_features].boxplot(ax=axes[1])
        axes[1].set_title('Texture Features')
        axes[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # 4. Feature importance (variance-based)
    feature_variance = df[feature_cols].var().sort_values(ascending=False)
    
    plt.figure(figsize=(12, 6))
    top_features = feature_variance.head(15)
    top_features.plot(kind='bar')
    plt.title('Top 15 Features by Variance', fontsize=14, fontweight='bold')
    plt.xlabel('Features')
    plt.ylabel('Variance')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
    
else:
    print("⚠️  No data available for visualization")
    print("Run the batch processing cell first to generate data")

## 🎯 Summary and Best Practices

### Key Takeaways:
1. **Single vs Batch**: Use `extract_features()` for individual cases, `extract_batch()` for multiple cases
2. **Configuration**: Choose appropriate preset configs or create custom ones
3. **Parallel Processing**: Enable parallel processing for large datasets
4. **Feature Analysis**: Always analyze feature correlations and distributions
5. **Quality Control**: Check for missing values and outliers

### Common Pitfalls:
- ⚠️ Mismatched image and mask dimensions
- ⚠️ Incorrect file paths or permissions
- ⚠️ Memory issues with large 3D images
- ⚠️ Inconsistent preprocessing across batches

### Next Steps:
- 🔄 Combine with segmentation results from `onem_segment`
- 🔗 Use features for machine learning models
- 📊 Perform feature selection and dimensionality reduction
- 🧪 Validate features on independent test sets